# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os
import time
from collections import deque
from dataclasses import asdict

from unityagents import UnityEnvironment
import mlflow
import numpy as np
import torch

from agents import MultiAgentDDPG
from experiment import RunExperiments, RunConfig

/home/akio/.pyenv/versions/3.6.12/envs/drlnd/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
# Global constants for the project.
# Number of episodes to train all agents.
NUM_EPISODES = 5000

# Consecutive runs to average the scores.
RESULT_WINDOW = 100

# Minimum average score over the window to complete the project.
MIN_AVERAGE_SCORE = 0.5

In [3]:
experiments_to_run = RunExperiments(
    learn_steps=[1, 2, 4],
    sync_steps=[8, 16, 32],
    batch_sizes=[32, 64],
    gammas=[0.98, 0.99],
    epsilon_decays=[0.997, 0.999]
)
NUM_EXPERIMENTS = 8

In [4]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [7]:
def train(num_episodes, env, agent):
    all_scores = []
    score_window = deque(maxlen=RESULT_WINDOW)
    solved = False
    best_mean_score = 0
    
    for episode_i in range(num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]      # Reset the environment    
        states = env_info.vector_observations                  # Get the current state (for each agent)
        scores = np.zeros(num_agents)                          # Initialize the score (for each agent)
        done = False

        while not done:
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]           # Send all actions to tne environment
            next_states = env_info.vector_observations         # Get next state (for each agent)
            rewards = env_info.rewards                         # Get reward (for each agent)
            dones = env_info.local_done                        # See if episode finished

            agent.step(states, actions, rewards, next_states, dones)

            states = next_states                               # Roll over states to next time step
            scores += env_info.rewards                         # Update the score (for each agent)
            
            if np.any(dones):                                  # Exit loop if episode finished
                done = True
        
        agent.episode_finished()
        max_score = np.max(scores)
        score_window.append(max_score)
        all_scores.append(max_score)
        
        current_mean = np.mean(score_window)
        if current_mean >= MIN_AVERAGE_SCORE:
            solved = True
        
        if current_mean > best_mean_score:
            best_mean_score = current_mean
        mlflow.log_metric("current_mean", current_mean, step=episode_i)
        mlflow.log_metric("current_epsilon", agent.agents[0].epsilon, step=episode_i)
        print(f"\r  Current mean score on episode {episode_i}: {current_mean}", end="")
        
    return all_scores, solved, best_mean_score, current_mean

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
%%time

experiment_scores = {}
mlflow.set_experiment("tennis-maddpg")
for i, config in enumerate(experiments_to_run.get_configs(NUM_EXPERIMENTS)):
    with mlflow.start_run(run_name=f"Config {i}"):
        start_time = time.time()
        print(f"Running experiment {i} with config: {config}")
        mlflow.log_params(asdict(config))
        
        agent = MultiAgentDDPG(
            num_agents,
            state_size,
            action_size,
            gamma=config.gamma,
            learn_step=config.learn_step,
            sync_step=config.sync_step,
            epsilon_decay=config.epsilon_decay,
            batch_size=config.batch_size
        )

        scores, solved, best_mean_score, last_mean_score = train(NUM_EPISODES, env, agent)
        experiment_scores[config] = scores
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"  Ran experiment in {elapsed} seconds.")
        print(f"  Best mean score of {best_mean_score}.")
        print(f"  Latest mean score of {last_mean_score}.")
        mlflow.log_metric("best_mean_score", best_mean_score)
        mlflow.log_metric("last_mean_score", last_mean_score)

        if solved:
            print("  Solved.")
            os.makedirs(f"./models/{i}/", exist_ok=True)
            torch.save(agent.critic_local.state_dict(), f"./models/{i}/critic.pth")
            for k, actor in enumerate(agent.agents):
                torch.save(actor.actor_local.state_dict(), f"./models/{i}/actor-{k}.pth")
            break

Running experiment 0 with config: RunConfig(learn_step=1, sync_step=8, batch_size=32, gamma=0.99, epsilon_decay=0.999)
  Current mean score on episode 3372: 0.009000000171363354

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 8))
for c, scores in experiment_scores.items():
    plt.plot(scores, label=c, alpha=0.7)
    
plt.title("Experiment Results")
plt.ylabel("Average Score")
plt.xlabel("Episode")
plt.legend(bbox_to_anchor=(0.5, -0.08))
plt.grid()
plt.show()

In [ ]:
agent = MultiAgentDDPG(
    num_agents,
    state_size,
    action_size,
    epsilon_start=0.0,
    min_epsilon=0.0)
agent.critic_local.load_state_dict(torch.load(f"./models/{i}/critic.pth"))
agent.critic_target.load_state_dict(torch.load(f"./models/{i}/critic.pth"))
for k, actor in enumerate(agent.agents):
    actor.actor_local.load_state_dict(torch.load(f"./models/{i}/actor-{k}.pth"))
    actor.actor_target.load_state_dict(torch.load(f"./models/{i}/actor-{k}.pth"))

done = False
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations
final_score = np.zeros(num_agents)

while not done:
    actions = agent.act(states)
    env_info = env.step(actions)[brain_name]
    next_states = env_info.vector_observations
    rewards = env_info.rewards
    dones = env_info.local_done

    states = next_states
    final_score += env_info.rewards

    if np.any(dones):
        done = True

print(f"Final score: {final_score}")

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```